In [1]:
import time
from datetime import timedelta

import html
import re

import numpy as np
import pandas as pd
import xml.etree.ElementTree as et

import dask
import dask.dataframe as dd
import dask.bag as bd
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

### Note
- Some of the files were not complete use the s1_find_incomp_chekstyle_xml.ipnb to find them and remove them
- and the empty ones as well

In [2]:
##########################################################
#Step 1. Format 'pmd_rules_results.xml' from Linux terminal#
##########################################################
# -'pmd_rules_results.xml' is not properly formated
# -We used linux format command to format 'pmd_rules_results.xml' to save it in 'pmd_rules_results_fmt.xml'
# $  xmllint --format checkstyle-result0.xml > checkstyle-result_fmt0.xml
##############################################################################
#Step 2. Remove some lines in 'checkstyle-result_fmt0.xml' from Linux terminal#
##############################################################################
# remove lines 1 to 2 and the last line of the 'checkstyle-result_fmt0.xml'
# MacOS $  sed -i '' '1,2d;$d' checkstyle-result_fmt0.xml 
# Linux  $   sed -i '1,2d;$d' checkstyle-result_fmt0.xml            
# blocksize breaks it into partitions
# Use '</file>\n' as linedelimiter
spotbugs_bd = bd.read_text('spotbugsxmlreports_xml/spotbugsXml*.xml', linedelimiter='</BugInstance>')


In [3]:
# check to see number of partitions
# find the number of partitions
#pmd_rules_result_bg.npartitions # or
spotbugs_bd

dask.bag<bag-from-delayed, npartitions=577>

In [4]:
# look at the first row
spotbugs_bd.take(1)

('<?xml version="1.0" encoding="utf-8"?>\n<BugCollection sequence=\'0\' release=\'\' analysisTimestamp=\'1638750455136\' version=\'4.5.0\' timestamp=\'1638748206174\'><Project projectName=\'my-javacodeanalysis-app0\'><Jar>/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/SpotBug1/mvn_apps/my-javacodeanalysis-app0/target/classes</Jar><SrcDir>/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/SpotBug1/mvn_apps/my-javacodeanalysis-app0/src/main/java</SrcDir><WrkDir>/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/SpotBug1/mvn_apps/my-javacodeanalysis-app0/target</WrkDir></Project><BugInstance instanceOccurrenceNum=\'0\' instanceHash=\'ce90bd5e82bd4d0d11cd96245d7fcadf\' rank=\'18\' abbrev=\'UrF\' category=\'PERFORMANCE\' priority=\'2\' type=\'URF_UNREAD_FIE

In [5]:
# look at the first 50 row
#spotbugs_bd.take(50)

### We are interested in getting the 
- 'xml header'
- BugCollection
- Project
- BugCategory

In [6]:
#@@#
# replace all the '<xml ...>' header with an empty string so that we can access it using xml.etree.ElementTree
spotbugs_bd = spotbugs_bd.map(lambda line: re.sub(r'\s*<\s*\?xml.*\?>', '', line))

In [7]:
#@@#
# look at the first row
spotbugs_bd.take(1)

("\n<BugCollection sequence='0' release='' analysisTimestamp='1638750455136' version='4.5.0' timestamp='1638748206174'><Project projectName='my-javacodeanalysis-app0'><Jar>/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/SpotBug1/mvn_apps/my-javacodeanalysis-app0/target/classes</Jar><SrcDir>/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/SpotBug1/mvn_apps/my-javacodeanalysis-app0/src/main/java</SrcDir><WrkDir>/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/SpotBug1/mvn_apps/my-javacodeanalysis-app0/target</WrkDir></Project><BugInstance instanceOccurrenceNum='0' instanceHash='ce90bd5e82bd4d0d11cd96245d7fcadf' rank='18' abbrev='UrF' category='PERFORMANCE' priority='2' type='URF_UNREAD_FIELD' instanceOccurrenceMax='0'><ShortMessage>Unread field</Short

In [8]:
# Drop BugCollection
#https://stackoverflow.com/questions/37684722/regex-matching-specific-html-tags
# replace all the '<BugCollection>' or '</BugCollection>' with empty string
#spotbugs_bd = spotbugs_bd.map(lambda line: re.sub(r'^\s*<\s*BugCollection.*>$|^<\s*/BugCollection>$', '', line))
spotbugs_bd = spotbugs_bd.map(lambda line: re.sub(r'\n\s*<BugCollection\s*.*?>|</BugCollection>', '', line))

In [9]:
# look at the first row
spotbugs_bd.take(1)

("<Project projectName='my-javacodeanalysis-app0'><Jar>/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/SpotBug1/mvn_apps/my-javacodeanalysis-app0/target/classes</Jar><SrcDir>/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/SpotBug1/mvn_apps/my-javacodeanalysis-app0/src/main/java</SrcDir><WrkDir>/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/SpotBug1/mvn_apps/my-javacodeanalysis-app0/target</WrkDir></Project><BugInstance instanceOccurrenceNum='0' instanceHash='ce90bd5e82bd4d0d11cd96245d7fcadf' rank='18' abbrev='UrF' category='PERFORMANCE' priority='2' type='URF_UNREAD_FIELD' instanceOccurrenceMax='0'><ShortMessage>Unread field</ShortMessage><LongMessage>Unread field: Code_10203353_10202514_4576_2.res</LongMessage><Class classname='Code_10203353_102

In [10]:
#@@# Drop Project
#https://stackoverflow.com/questions/20045955/regex-pattern-in-python-for-parsing-html-title-tags
# replace all the '<Project>' and '</Project>' with empty string
# <Project>...</Project>
spotbugs_bd = spotbugs_bd.map(lambda line: re.sub(r'<Project\s*.*?>(.+?)</Project>', '', line))
#spotbugs_bd = spotbugs_bd.map(lambda line: re.sub(r'\n\s*<Project\s*.*?>(.+?)</Project>\n', '', line))

In [11]:
# look at the first row
spotbugs_bd.take(1)

("<BugInstance instanceOccurrenceNum='0' instanceHash='ce90bd5e82bd4d0d11cd96245d7fcadf' rank='18' abbrev='UrF' category='PERFORMANCE' priority='2' type='URF_UNREAD_FIELD' instanceOccurrenceMax='0'><ShortMessage>Unread field</ShortMessage><LongMessage>Unread field: Code_10203353_10202514_4576_2.res</LongMessage><Class classname='Code_10203353_10202514_4576_2' primary='true'><SourceLine classname='Code_10203353_10202514_4576_2' start='1' end='2' sourcepath='Code_10203353_10202514_4576_2.java' sourcefile='Code_10203353_10202514_4576_2.java'><Message>At Code_10203353_10202514_4576_2.java:[lines 1-2]</Message></SourceLine><Message>In class Code_10203353_10202514_4576_2</Message></Class><Field isStatic='false' classname='Code_10203353_10202514_4576_2' signature='Ljava/lang/String;' name='res' primary='true'><SourceLine classname='Code_10203353_10202514_4576_2' sourcepath='Code_10203353_10202514_4576_2.java' sourcefile='Code_10203353_10202514_4576_2.java'><Message>In Code_10203353_10202514_4

In [12]:
#filter out only <BugInstance ...
# Regex can also be applied here
spotbugs_bd = spotbugs_bd.filter(lambda line: line.find('<BugInstance') >= 0)

In [14]:
# look at the first row
spotbugs_bd.take(1)

("<BugInstance instanceOccurrenceNum='0' instanceHash='ce90bd5e82bd4d0d11cd96245d7fcadf' rank='18' abbrev='UrF' category='PERFORMANCE' priority='2' type='URF_UNREAD_FIELD' instanceOccurrenceMax='0'><ShortMessage>Unread field</ShortMessage><LongMessage>Unread field: Code_10203353_10202514_4576_2.res</LongMessage><Class classname='Code_10203353_10202514_4576_2' primary='true'><SourceLine classname='Code_10203353_10202514_4576_2' start='1' end='2' sourcepath='Code_10203353_10202514_4576_2.java' sourcefile='Code_10203353_10202514_4576_2.java'><Message>At Code_10203353_10202514_4576_2.java:[lines 1-2]</Message></SourceLine><Message>In class Code_10203353_10202514_4576_2</Message></Class><Field isStatic='false' classname='Code_10203353_10202514_4576_2' signature='Ljava/lang/String;' name='res' primary='true'><SourceLine classname='Code_10203353_10202514_4576_2' sourcepath='Code_10203353_10202514_4576_2.java' sourcefile='Code_10203353_10202514_4576_2.java'><Message>In Code_10203353_10202514_4

### Use ElementTree to get all the attrbutes and text in xmlor html tags

In [15]:
# This will give us just the attributes in the BugInstance tag: <BugInstance ...>
spotbugs_bd1 = spotbugs_bd.map(lambda row: et.fromstring(row).attrib)

In [16]:
spotbugs_bd1.take(1)

({'instanceOccurrenceNum': '0',
  'instanceHash': 'ce90bd5e82bd4d0d11cd96245d7fcadf',
  'rank': '18',
  'abbrev': 'UrF',
  'category': 'PERFORMANCE',
  'priority': '2',
  'type': 'URF_UNREAD_FIELD',
  'instanceOccurrenceMax': '0'},)

In [17]:
# Convert text in tag to key/value
def update_dict_key_wt_text_as_value(tag, value, _dict):
    if tag == 'ShortMessage': 
        _dict['shortMsg'] = value
    if tag == 'LongMessage': 
        _dict['longMsg'] = value
    if tag == 'Message': 
        _dict['msg'] = value
        
    return _dict

In [18]:
# Add extra key value to dict if they do not exixt
def add_key_val_if_not_exist(_dict):
    _dict
    keys = [
        'abbrev',
        'category',
        'classname',
        'end', 
        'endBytecode',
        'instanceHash',
        'instanceOccurrenceMax',
        'instanceOccurrenceNum',
        'isStatic',
        'longMsg',
        'msg',
        'name',
        'primary',
        'priority',
        'signature', 
        'shortMsg',
        'sourcepath', 
        'sourcefile',
        'start',
        'startBytecode',
        'type',
        'rank',
        'role',
     
    ]
    value=''
    for key in keys:
        if key not in _dict:
            _dict[key] = value
    return _dict

In [19]:
#https://stackoverflow.com/questions/43921237/access-nested-children-in-xml-file-parsed-with-elementtree
def content(root):
    list_of_dicts = []
    try:
        for child in root.iter():
            # update dict with text as values
            _dict = update_dict_key_wt_text_as_value(child.tag, child.text, child.attrib)
            # update dict with keys that do not exist
            _dict = add_key_val_if_not_exist(_dict)
            list_of_dicts.append(_dict)
    except ParseError:
        # pass an empty dict eo update all keys with empty values
        list_of_dicts.append(add_key_val_if_not_exist({}))

    return list_of_dicts

In [20]:
# This will give us just the text in the BugInstance tag: <BugInstance ...>
spotbugs_bd = spotbugs_bd.map(lambda row: content(et.fromstring(row)))

In [237]:
spotbugs_bd.take(1)

([{'instanceOccurrenceNum': '0',
   'instanceHash': 'ce90bd5e82bd4d0d11cd96245d7fcadf',
   'rank': '18',
   'abbrev': 'UrF',
   'category': 'PERFORMANCE',
   'priority': '2',
   'type': 'URF_UNREAD_FIELD',
   'instanceOccurrenceMax': '0',
   'classname': '',
   'end': '',
   'endBytecode': '',
   'isStatic': '',
   'longMsg': '',
   'msg': '',
   'name': '',
   'primary': '',
   'signature': '',
   'shortMsg': '',
   'sourcepath': '',
   'sourcefile': '',
   'start': '',
   'startBytecode': '',
   'role': ''},
  {'shortMsg': 'Unread field',
   'abbrev': '',
   'category': '',
   'classname': '',
   'end': '',
   'endBytecode': '',
   'instanceOccurrenceMax': '',
   'instanceOccurrenceNum': '',
   'instanceHash': '',
   'isStatic': '',
   'longMsg': '',
   'msg': '',
   'name': '',
   'primary': '',
   'priority': '',
   'signature': '',
   'sourcepath': '',
   'sourcefile': '',
   'start': '',
   'startBytecode': '',
   'type': '',
   'rank': '',
   'role': ''},
  {'longMsg': 'Unread f

In [21]:
# Gets Rid of the empty partitions in Bags
def cull_empty_partitions(bag):
    """
    When bags are created by filtering or grouping from a different bag,
    it retains the original bag's partition count, even if a lot of the
    partitions become empty.
    Those extra partitions add overhead, so it's nice to discard them.
    This function drops the empty partitions.
    """
    bag = bag.persist()
    def get_len(partition):
        # If the bag is the result of bag.filter(),
        # then each partition is actually a 'filter' object,
        # which has no __len__.
        # In that case, we must convert it to a list first.
        if hasattr(partition, '__len__'):
            return len(partition)
        return len(list(partition))
    partition_lengths = bag.map_partitions(get_len).compute()

    # Convert bag partitions into a list of 'delayed' objects
    lengths_and_partitions = zip(partition_lengths, bag.to_delayed())

    # Drop the ones with empty partitions
    partitions = (p for l,p in lengths_and_partitions if l > 0)

    # Convert from list of delayed objects back into a Bag.
    return dask.bag.from_delayed(partitions)

In [23]:
spotbugs_bd = cull_empty_partitions(spotbugs_bd)

In [24]:
# replace all the '\n' with empty string so that we can access it using xml.etree.ElementTree
#chk_rules_result_bd = chk_rules_result_bd.map(lambda line: re.sub(r'(\n$)', '@@anyi@@', line))
#chk_rules_result_bd = chk_rules_result_bd.map(lambda line: re.sub(r'(\r\n$)', "\n<file name=''>\n<error line='' severity='',message='' source=''/>\n</file>", line))

## Start a Dask cluster using SLURM jobs as workers.

There are a couple of things we need to configure here:

disabling the mechanism to write on disk when workers run out of memory,
memory, CPUs, maximum time and number of workers per SLURM job,
dask folders for log files and workers data.
We recommend putting the log folder and workers data folders in your /nesi/nobackup/<project_code> folder, most indicated for temporary files (see NeSI File Systems and Quotas).

All of these options can be set in configuration files, see Dask configuration and Dask jobqueue configuration for more information.

In [25]:
#http://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html
dask.config.set(
    {
        "distributed.worker.memory.target": False,  # avoid spilling to disk
        "distributed.worker.memory.spill": False,  # avoid spilling to disk
    }
)
cluster = SLURMCluster(
    cores=10, #cores=24, # we set each job to have 1 Worker, each using 10 cores (threads) and 8 GB of memory
    processes=2,
    memory="8GiB",
    walltime="0-30:30",# walltime="0-00:30",
    log_directory="../dask/logs",  # folder for SLURM logs for each worker
    local_directory="../dask",  # folder for workers data
)

In [26]:
#cluster.adapt(minimum_jobs=20, maximum_jobs=200)
cluster.adapt(minimum_jobs=10, maximum_jobs=200)
client = Client(cluster)
client

Client Scheduler: tcp://192.168.94.166:36852 Dashboard: http://192.168.94.166:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [27]:
######START######

## Flatten

In [28]:
spotbugs_bd = spotbugs_bd.flatten()

## Convert Bags to Dataframes

In [29]:
#convert from bags to dataframe
df = spotbugs_bd.to_dataframe()

In [31]:
#df = spotbugs_bd.to_dataframe()
#df = files_rows_bd.to_dataframe(meta={'name':'str', 'line':'int', 'severity':'str', 'message':'str', 'source':'str'})

In [32]:
df

,instanceOccurrenceNum,instanceHash,rank,abbrev,category,priority,type,instanceOccurrenceMax,classname,end,endBytecode,isStatic,longMsg,msg,name,primary,signature,shortMsg,sourcepath,sourcefile,start,startBytecode,role
npartitions=577,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [33]:
df.columns

Index(['instanceOccurrenceNum', 'instanceHash', 'rank', 'abbrev', 'category',
       'priority', 'type', 'instanceOccurrenceMax', 'classname', 'end',
       'endBytecode', 'isStatic', 'longMsg', 'msg', 'name', 'primary',
       'signature', 'shortMsg', 'sourcepath', 'sourcefile', 'start',
       'startBytecode', 'role'],
      dtype='object')

In [34]:
df.dtypes

instanceOccurrenceNum    object
instanceHash             object
rank                     object
abbrev                   object
category                 object
priority                 object
type                     object
instanceOccurrenceMax    object
classname                object
end                      object
endBytecode              object
isStatic                 object
longMsg                  object
msg                      object
name                     object
primary                  object
signature                object
shortMsg                 object
sourcepath               object
sourcefile               object
start                    object
startBytecode            object
role                     object
dtype: object

In [35]:
df.npartitions

577

In [36]:
#df.head()

In [37]:
df.tail()

,instanceOccurrenceNum,instanceHash,rank,abbrev,category,priority,type,instanceOccurrenceMax,classname,end,...,msg,name,primary,signature,shortMsg,sourcepath,sourcefile,start,startBytecode,role
800,,,,,,,,,Code_997269_332079_2894_1,,...,,,,,,Code_997269_332079_2894_1.java,Code_997269_332079_2894_1.java,,,
801,,,,,,,,,,,...,In Code_997269_332079_2894_1.java,,,,,,,,,
802,,,,,,,,,,,...,Field Code_997269_332079_2894_1.hexArray,,,,,,,,,
803,,,,,,,,,Code_997269_332079_2894_1,2,...,,,true,,,Code_997269_332079_2894_1.java,Code_997269_332079_2894_1.java,2,1801,
804,,,,,,,,,,,...,At Code_997269_332079_2894_1.java:[line 2],,,,,,,,,


In [38]:
#df.head().compute()
df.head()

,instanceOccurrenceNum,instanceHash,rank,abbrev,category,priority,type,instanceOccurrenceMax,classname,end,...,msg,name,primary,signature,shortMsg,sourcepath,sourcefile,start,startBytecode,role
0,0,ce90bd5e82bd4d0d11cd96245d7fcadf,18,UrF,PERFORMANCE,2,URF_UNREAD_FIELD,0,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,Unread field,,,,,
2,,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,Code_10203353_10202514_4576_2,,...,,,true,,,,,,,
4,,,,,,,,,Code_10203353_10202514_4576_2,2,...,,,,,,Code_10203353_10202514_4576_2.java,Code_10203353_10202514_4576_2.java,1,,


In [39]:
df.shape[0].compute()

534978

## Save all the codes from the posts into a CSV file

In [40]:
## Save all the codes from the posts into a JSON file
#violations_rows_bd.to_textfiles('pmdcodesnippets_json/PMDJavaCodeSnippets*.json.gz', , encoding='utf-8')
#violations_rows_bd.to_textfiles('pmdcodesnippets_json/PMDJavaCodeSnippets*.json', encoding='utf-8')
## Save all the codes from the posts into a CSV file
# Save to a CSV file
_ = df.to_csv('spotbugxmlreports_csv/spotbugReport*.csv', sep=',', index=False)